In [1]:
## Importing all lib
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [2]:
## Loading Train Model
model=load_model('model.h5')

## Loading Scaler,Encoder pickel
with open('ohe_geo.pkl','rb') as file:
    ohe_geo=pickle.load(file)
with open('le_gender.pkl','rb') as file:
    le_gender=pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [3]:
# Testing with Custom IP data
input_data = {
    'CreditScore': 550,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 35,
    'Tenure': 3,
    'Balance': 75000,
    'NumOfProducts': 3,
    'HasCrCard': 1,
    'IsActiveMember': 2,
    'EstimatedSalary': 60000
}

"""input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}"""

"input_data = {\n    'CreditScore': 600,\n    'Geography': 'France',\n    'Gender': 'Male',\n    'Age': 40,\n    'Tenure': 3,\n    'Balance': 60000,\n    'NumOfProducts': 2,\n    'HasCrCard': 1,\n    'IsActiveMember': 1,\n    'EstimatedSalary': 50000\n}"

In [4]:
## OHE in Geography
geo_encoded=ohe_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\Projects\Bank Customer's Churn Prediction\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df=pd.DataFrame([input_data])
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df['France']=input_df['Geography_France']
input_df['Germany']=input_df['Geography_Germany']
input_df['Spain']=input_df['Geography_Spain']
input_df=input_df.drop('Geography_France',axis=1)
input_df=input_df.drop('Geography_Germany',axis=1)
input_df=input_df.drop('Geography_Spain',axis=1)
input_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,550,Female,35,3,75000,3,1,2,60000,1.0,0.0,0.0


In [6]:
## LE in Gender (0-Female,1-Male)
input_df['Gender']=le_gender.transform(input_df['Gender'])
input_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,550,0,35,3,75000,3,1,2,60000,1.0,0.0,0.0


In [7]:
## Scaling the df
input_scaled=scaler.transform(input_df)
input_scaled

array([[-1.05487171, -1.09499335, -0.37056859, -0.69539349, -0.01764635,
         2.53355998,  0.64920267,  2.97546756, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [8]:
## Predicting Churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


array([[0.5024284]], dtype=float32)

In [9]:
prediction_prob=prediction[0][0]
prediction_prob

np.float32(0.5024284)

In [10]:
if prediction_prob > 0.5:
    print("The Customer is likely to Churn.")
else :
    print("The Customer is not likely to Churn.")

The Customer is likely to Churn.
